In [1]:
! pip install xlrd==2.0.1 openpyxl


In [35]:
import os
import pandas as pd
import numpy as np

def read_excel_file(file_path, sheet_name):
    """Helper function to read Excel files with different engines based on file extension."""
    try:
        if file_path.endswith(".xls"):
            return pd.read_excel(file_path, sheet_name=sheet_name, engine='xlrd', skiprows=1)
        else:
            return pd.read_excel(file_path, sheet_name=sheet_name, skiprows=1)
    except Exception as e:
        print(f"Failed to read '{sheet_name}' from {file_path}: {e}")
        return None

def extract_and_combine_data(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    data_by_year = {}

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".xls") or file_name.endswith(".xlsx"):
            year = file_name.split()[0]
            
            try:
                year_int = int(year)
                if year_int not in [2011, 2016]:
                    continue
            except ValueError:
                print(f"Could not extract a valid year from {file_name}. Skipping this file.")
                continue
            
            file_path = os.path.join(input_folder, file_name)
            
            # Load data from 'Additional Measure Data' sheet
            df = read_excel_file(file_path, 'Additional Measure Data')
            if df is None:
                continue

            # Load data from 'Ranked Measure Data' or 'Select Measure Data' sheet
            df2 = read_excel_file(file_path, 'Ranked Measure Data')
            if df2 is None:
                df2 = read_excel_file(file_path, 'Select Measure Data')
            if df2 is None:
                continue

            # Identify relevant columns in df
            missing_columns = []
            column_name = df.columns[df.columns.isin(['% Limited Access to Healthy Foods', '% Limited Access', '% Healthy Food'])].tolist()
            diabetes_column = df.columns[df.columns.isin(['% Diabetic', '% diabetic', '% Adults with Diabetes', 'Diabetes'])].tolist()
            rural_column = df.columns[df.columns.isin(['% Rural', '% rural', 'Rural'])].tolist()
            income_column = df.columns[df.columns.isin(['Household Income', 'Median Household Income'])].tolist()
            region_column = df.columns[df.columns.isin(['County', 'Parish', 'Borough'])].tolist()
            african_american_column = df.columns[df.columns.isin(['African American', '% African American'])].tolist()
            asian_column = df.columns[df.columns.isin(['Asian', '% Asian'])].tolist()
            hispanic_column = df.columns[df.columns.isin(['Hispanic', '% Hispanic'])].tolist()
            drinking_2011_column = df.columns[df.columns.isin(['% Binge Drinking'])].tolist()
            physical_2011_column = df.columns[df.columns.isin(['Physical Inactivity', '% Physically Inactive'])].tolist()

            # Ensure that each list is not empty and get the first element, or create NaN columns if not found
            # column_name = column_name[0] if column_name else missing_columns.append('% Limited Access')
            diabetes_column = diabetes_column[0] if diabetes_column else missing_columns.append('% Diabetic')
            rural_column = rural_column[0] if rural_column else missing_columns.append('% Rural')
            income_column = income_column[0] if income_column else missing_columns.append('Household Income')
            region_column = region_column[0] if region_column else missing_columns.append('Region (County/Parish/Borough)')
            african_american_column = african_american_column[0] if african_american_column else missing_columns.append('African American')
            asian_column = asian_column[0] if asian_column else missing_columns.append('Asian')
            hispanic_column = hispanic_column[0] if hispanic_column else missing_columns.append('Hispanic')

            # Create missing columns for drinking and physical inactivity, filled with NaN
            if drinking_2011_column:
                drinking_2011_column = drinking_2011_column[0]
            else:
                df['% Binge Drinking'] = np.nan
                drinking_2011_column = '% Binge Drinking'

            if column_name:
                column_name = column_name[0]
            else:
                df['% Limited Access'] = np.nan
                column_name = '% Limited Access'

            if physical_2011_column:
                physical_2011_column = physical_2011_column[0]
            else:
                df['Physical Inactivity'] = np.nan
                physical_2011_column = 'Physical Inactivity'

            # Identify relevant columns in df2
            mentally_unhealthy_column = df2.columns[df2.columns.isin(['Average Number of Mentally Unhealthy Days', 'Mentally Unhealthy Days'])].tolist()
            # food_env_column = df2.columns[df2.columns.isin(['Food Environment Index'])].tolist()
            # income_ratio_column = df2.columns[df2.columns.isin(['Income Ratio'])].tolist()
            obese_column = '% Obese'
            drinking_2016_column = '% Excessive Drinking'
            primarycare_column = 'PCP Ratio'
            poorhealth_column = '% Fair/Poor'
            physical_2016_column = df2.columns[df2.columns.isin(['% Physically Inactive'])].tolist()


            mentally_unhealthy_column = mentally_unhealthy_column[0] if mentally_unhealthy_column else missing_columns.append('Mentally Unhealthy Days')
            # food_env_column = food_env_column[0] if food_env_column else missing_columns.append('Food Environment Index')
            # income_ratio_column = income_ratio_column[0] if income_ratio_column else missing_columns.append('Income Ratio')
            obese_column = obese_column if obese_column else missing_columns.append('% Obese')

            if drinking_2016_column:
                drinking_2016_column = drinking_2016_column
            else:
                df2['% Excessive Drinking'] = np.nan
                drinking_2016_column = '% Excessive Drinking'

            if physical_2016_column:
                physical_2016_column = physical_2016_column[0]
            else:
                df2['% Physically Inactive'] = np.nan
                physical_2016_column = '% Physically Inactive'

            # If any other required column is missing, skip this file and print the file name and missing columns
            if missing_columns:
                print(f"File '{file_name}' is missing columns: {', '.join(missing_columns)}. Skipping this file.")
                continue
            # Print all columns for debugging
            print(f"Columns in {file_name} - 'Additional Measure Data': {df.columns.tolist()}")
            print(f"Columns in {file_name} - 'Ranked Measure Data'/'Select Measure Data': {df2.columns.tolist()}")

            print(f"Merging on columns: FIPS, {mentally_unhealthy_column}, {obese_column}, {drinking_2016_column}, {primarycare_column}, {poorhealth_column}, {physical_2016_column}")

            # If any other required column is missing, skip this file and print the file name and missing columns
            if missing_columns:
                print(f"File '{file_name}' is missing columns: {', '.join(missing_columns)}. Skipping this file.")
                continue

            # Merge and select relevant data
            merged_df = pd.merge(df, df2[['FIPS', mentally_unhealthy_column,  obese_column, drinking_2016_column, primarycare_column, poorhealth_column, physical_2016_column]], on='FIPS', how='inner')
            merged_df = merged_df[['FIPS', 'State', region_column, column_name, diabetes_column, rural_column, income_column, 
                                   african_american_column, asian_column, hispanic_column, drinking_2011_column, physical_2011_column, 
                                   mentally_unhealthy_column, obese_column, 
                                   drinking_2016_column, primarycare_column, poorhealth_column, physical_2016_column]]

            if year not in data_by_year:
                data_by_year[year] = merged_df
            else:
                data_by_year[year] = pd.concat([data_by_year[year], merged_df], ignore_index=True)

    # Write each year's data to a new CSV file
    for year, data in data_by_year.items():
        output_file_path = os.path.join(output_folder, f"{year}_Limited_Access_to_Healthy_Foods.csv")
        data.to_csv(output_file_path, index=False)
        print(f"Data for {year} saved to {output_file_path}")



In [36]:
input_folder = "County Health Rankings"  
output_folder = "health-regression-and-foodenv"
    
extract_and_combine_data(input_folder, output_folder)

Columns in 2011 County Health Ranking Wisconsin Data - v4.xls - 'Additional Measure Data': ['FIPS', 'State', 'County', 'Population', '< 18', '65 and over', 'African American', 'American Indian/ Alaskan Native', 'Asian', 'Native Hawaiian/ Other Pacific Islander', 'Hispanic', 'Population.1', '# not proficient in English', '% not proficient in English', 'Female', 'Rural', 'Diabetes', '95% CI - Low', '95% CI - High', 'Population.2', 'HIV rate', 'Sample Size', '% Binge Drinking', '95% CI - Low.1', '95% CI - High.1', 'Physical Inactivity', '95% CI - Low.2', '95% CI - High.2', 'MPH No.', 'Population.3', 'MPH Rate', 'MPH Ratio', 'Household Income', '95% CI - Low.3', '95% CI - High.3', '# high housing costs', 'Households', '% high housing costs', '% Free lunch', 'Population.4', 'Illiteracy', '95% CI - Low.4', '95% CI - High.4', 'Population.5', 'Liquor Stores', 'Liquor Store Rate', 'No. Drive Alone', 'Workers', '% Drive Alone', '% Limited Access']
Columns in 2011 County Health Ranking Wisconsin 